In [245]:
#dependencies
import pandas as pd

In [252]:
#import csv and read in data
passenger_df = pd.read_csv("../Data_Project/combined.csv")

In [253]:
passenger_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [254]:
passenger_df = passenger_df.drop(['Cabin','Ticket'], axis=1)

In [256]:
passenger_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [257]:
#Combine SibSP and Parch into new column 'Family'
passenger_df['Family'] = passenger_df['SibSp'] + passenger_df['Parch']

In [259]:
#Drop Name, Fare, Embark ID
new_passenger_df = passenger_df.drop(['PassengerId','Name', 'SibSp', 'Parch', 'Fare', 'Embarked'], axis=1)
new_passenger_df.head()

,Survived,Pclass,Sex,Age,Family
0,0,3,male,22.0,1
1,1,1,female,38.0,1
2,1,3,female,26.0,0
3,1,1,female,35.0,1
4,0,3,male,35.0,0


In [280]:
# saving the dataframe
new_passenger_df.to_csv('new_passenger_df.csv')

In [281]:
new_passenger_df.describe()

,Survived,Pclass,Age,Family
count,1309.000000,1309.000000,1046.000000,1309.000000
mean,0.384263,2.294882,29.881138,0.883881
std,0.486606,0.837836,14.413493,1.583639
min,0.000000,1.000000,0.170000,0.000000
25%,0.000000,2.000000,21.000000,0.000000
50%,0.000000,3.000000,28.000000,0.000000
75%,1.000000,3.000000,39.000000,1.000000
max,1.000000,3.000000,80.000000,10.000000


In [282]:
features=list(new_passenger_df.columns)
features.remove('Survived')
X=new_passenger_df[features]
X['Age']=X['Age'].fillna(0)
y=new_passenger_df['Survived']
print(X.shape)
print(y.shape)

(1309, 4)
(1309,)


C:\Users\koryk\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [283]:
sex_df=pd.get_dummies(X['Sex'], drop_first=True)
#embarked_df=pd.get_dummies(X['Embarked'])
X=pd.concat([X, sex_df], axis=1)
X=X.drop(columns=['Sex'])
X.head()

,Pclass,Age,Family,male
0,3,22.0,1,1
1,1,38.0,1,0
2,3,26.0,0,0
3,1,35.0,1,0
4,3,35.0,0,1


In [284]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X, y)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(981, 4)
(981,)
(328, 4)
(328,)


In [266]:
X_train.head()

,Pclass,Age,Family,male
1114,3,21.0,0,1
216,3,27.0,0,0
407,2,3.0,2,1
1112,3,21.0,0,1
949,3,0.0,1,1


In [267]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [268]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(random_state=0)
rf.fit(X_train_scaled, y_train)
print(f'Train Score: {rf.score(X_train_scaled, y_train)}')
print(f'Test Score: {rf.score(X_test_scaled, y_test)}')

Train Score: 0.8858307849133538
Test Score: 0.7439024390243902


In [269]:
sorted(list(zip(rf.feature_importances_, X_train.columns)), reverse=True)

[(0.40737903418189253, 'Age'),
 (0.32005666005559164, 'male'),
 (0.15264667241586083, 'Pclass'),
 (0.11991763334665508, 'Family')]

In [270]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [271]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier(random_state=0)
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [272]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)
predictions

array([0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,

In [273]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

In [278]:
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

print("Confusion Matrix")
cm_df

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,180,38
Actual 1,42,68


In [279]:
# Displaying results

print(f"Accuracy Score : {rf.score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Accuracy Score : <bound method ClassifierMixin.score of RandomForestClassifier(random_state=0)>
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       218
           1       0.64      0.62      0.63       110

    accuracy                           0.76       328
   macro avg       0.73      0.72      0.72       328
weighted avg       0.75      0.76      0.75       328



In [276]:
features

['Pclass', 'Sex', 'Age', 'Family']

In [277]:
from sklearn import tree
import graphviz 

# Create a decision tree graph

titanic_data = tree.export_graphviz(
    model, out_file=None, 
    feature_names=X.features, 
    class_names=y,  
    filled=True, rounded=True,  
    #special_characters=True
) 

AttributeError: 'DataFrame' object has no attribute 'features'